In [26]:
#CRAWLING DATA FROM YAHOO MOVIES
import requests
import re
from bs4 import BeautifulSoup

Y_MOVIE_URL = "https://movies.yahoo.com.tw/movieinfo_main/"

# YOUR CODE HERE! 
# IMPLEMENTIG YAHOO MOVIES CRAWLER
class MovieCrawler(object):

    def __init__(self):
        self.session = requests.Session()
        
    def get_movies(self, page_url):
        movies = []
          # with open(page_url,'r') as f:
          #   contents = f.read()
        for i in range(2,3):
            response = self.session.get(page_url+str(i))#+"?page="+str(i))
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')
            movie_list = soup.find_all('div', id='content_l')
            # print(movie_list)
            for movie in movie_list:
                label_class = []
                # print(movie)
                try:
                    ch_name = movie.find('div', class_='movie_intro_info_r').h1#.text.strip()
                    try:
                        ch_name.span.decompose()
                    except:
                        pass
                    ch_name = ch_name.text.strip()
                    en_name = movie.find('div', class_='movie_intro_info_r').h3.text.strip()
                    release_date = movie.find('div', class_='movie_intro_info_r').find('span',class_=None).text[5:].strip()#.span.text.split('上映日期：')[-1].strip()
                    labels = movie.find_all('div',class_ = 'level_name')
                    for label in labels:
                        try:
                            lb = label.find('a',class_='gabtn').text.strip()
                        except:
                            break
                        label_class.append(lb)
                    intro = movie.find('div', class_='gray_infobox_inner').span.text.strip().replace('\n','')
                    doc_id = i
                    movie_info = {
                                'doc_id' : doc_id,
                                'ch_name': ch_name,
                                'en_name': en_name,
                                'label[class]': label_class,
                                'release_date': release_date,
                                'intro': intro}
                    movies.append(movie_info)
                except:
                    print(i)
                    continue
        return movies


# # DO NOT MODIFY THE VARIABLES
crawler = MovieCrawler()
movie = crawler.get_movies(Y_MOVIE_URL)

# # THE RESULTS : AS THE FOLLOWING SECTION
# # {'ch_name', 'en_name', 'movie_url', 'release_date', 'intro'}
print(len(movie))
print(*movie, sep="\n")
#SAVING DATA INTO JSON FILE

1
{'doc_id': 2, 'ch_name': '玩命關頭', 'en_name': 'The Fast and the Furious', 'label[class]': ['動作', '劇情', '犯罪', '懸疑/驚悚'], 'release_date': '2001-10-13', 'intro': '唐米尼杜洛托是洛城街頭賽車界的老大哥，他身邊有一群忠心耿耿的手下，他白天忙著組裝高性能跑車，晚上則是開著他的愛車，動輒以一次一萬美元的賭注和別人軋車。布萊恩也渴望接受極速的挑戰，他對自己的駕駛技術很有信心，但是在旁觀者的眼中他只是一個菜鳥，他開了一輛超炫的跑車想和唐老大一較高下，也希望得到他的青睞，當比賽結束，布萊恩輸得一塌塗地之後，警方接獲風聲前來取締，布萊恩在無意間從一名心狠手辣的幫派份子強尼手中救了唐老大一命，於是他就被納入唐老大的權力核心，唐老大的妹妹蜜雅也對布萊恩產生好感，但是他們都不知道布萊恩其實是一名臥底警探。布萊恩滲入賽車圈的目的是調查一連串的卡車搶案，嫌犯都是開著跑車的蒙面人，警方和聯邦調查局希望能儘早逮到搶匪，以免卡車司機採取激烈的手段對這些搶匪進行報復行動，其中最有嫌疑的就是唐老大和強尼。正當唐老大和強尼形成水火不相容的情勢。布萊恩和唐老大兄妹的關係卻越來越深，他不但和唐老大結為好友，更忍不住對蜜雅產生好感，但是他也同時承受來自警方和ＦＢＩ的壓力，必須儘快查出誰才是真正的搶匪，他在天人交戰之際，在法律和友情之間，必須做出困難的決定。'}


In [128]:
#SAVING DATA INTO JSON FILE
import json
with open("./moviedata.json", "w",encoding = 'utf-8') as final:
    json.dump(listObj, final,ensure_ascii=False)

In [131]:
#APPENDING NEW DATA INTO FILE
import json
with open("./moviedata.json", "r",encoding='utf-8') as fp:
    listObj = json.load(fp)
#     for m in movie:
#         listObj.append(m)
print(len(listObj))
print(listObj[1])
# with open("./hw2_1.json", "w",encoding = 'utf-8') as final:
#     json.dump(intro_all, final,ensure_ascii=False)

12238
{'doc_id': 2, 'ch_name': '玩命關頭', 'en_name': 'The Fast and the Furious', 'label[class]': ['動作', '劇情', '犯罪', '懸疑/驚悚'], 'release_date': '2001-10-13', 'intro': '唐米尼杜洛托是洛城街頭賽車界的老大哥，他身邊有一群忠心耿耿的手下，他白天忙著組裝高性能跑車，晚上則是開著他的愛車，動輒以一次一萬美元的賭注和別人軋車。布萊恩也渴望接受極速的挑戰，他對自己的駕駛技術很有信心，但是在旁觀者的眼中他只是一個菜鳥，他開了一輛超炫的跑車想和唐老大一較高下，也希望得到他的青睞，當比賽結束，布萊恩輸得一塌塗地之後，警方接獲風聲前來取締，布萊恩在無意間從一名心狠手辣的幫派份子強尼手中救了唐老大一命，於是他就被納入唐老大的權力核心，唐老大的妹妹蜜雅也對布萊恩產生好感，但是他們都不知道布萊恩其實是一名臥底警探。布萊恩滲入賽車圈的目的是調查一連串的卡車搶案，嫌犯都是開著跑車的蒙面人，警方和聯邦調查局希望能儘早逮到搶匪，以免卡車司機採取激烈的手段對這些搶匪進行報復行動，其中最有嫌疑的就是唐老大和強尼。正當唐老大和強尼形成水火不相容的情勢。布萊恩和唐老大兄妹的關係卻越來越深，他不但和唐老大結為好友，更忍不住對蜜雅產生好感，但是他也同時承受來自警方和ＦＢＩ的壓力，必須儘快查出誰才是真正的搶匪，他在天人交戰之際，在法律和友情之間，必須做出困難的決定。'}


In [111]:
import json
with open("./moviedata.json", "r",encoding = 'utf-8') as final:
    listObj = json.load(final)

In [130]:
for l in listObj:
    if 2 in l.values():
        print("true")
        print(l)

true
{'doc_id': 2, 'ch_name': '玩命關頭', 'en_name': 'The Fast and the Furious', 'label[class]': ['動作', '劇情', '犯罪', '懸疑/驚悚'], 'release_date': '2001-10-13', 'intro': '唐米尼杜洛托是洛城街頭賽車界的老大哥，他身邊有一群忠心耿耿的手下，他白天忙著組裝高性能跑車，晚上則是開著他的愛車，動輒以一次一萬美元的賭注和別人軋車。布萊恩也渴望接受極速的挑戰，他對自己的駕駛技術很有信心，但是在旁觀者的眼中他只是一個菜鳥，他開了一輛超炫的跑車想和唐老大一較高下，也希望得到他的青睞，當比賽結束，布萊恩輸得一塌塗地之後，警方接獲風聲前來取締，布萊恩在無意間從一名心狠手辣的幫派份子強尼手中救了唐老大一命，於是他就被納入唐老大的權力核心，唐老大的妹妹蜜雅也對布萊恩產生好感，但是他們都不知道布萊恩其實是一名臥底警探。布萊恩滲入賽車圈的目的是調查一連串的卡車搶案，嫌犯都是開著跑車的蒙面人，警方和聯邦調查局希望能儘早逮到搶匪，以免卡車司機採取激烈的手段對這些搶匪進行報復行動，其中最有嫌疑的就是唐老大和強尼。正當唐老大和強尼形成水火不相容的情勢。布萊恩和唐老大兄妹的關係卻越來越深，他不但和唐老大結為好友，更忍不住對蜜雅產生好感，但是他也同時承受來自警方和ＦＢＩ的壓力，必須儘快查出誰才是真正的搶匪，他在天人交戰之際，在法律和友情之間，必須做出困難的決定。'}


In [133]:
#INVERTED INDEX
import jieba
import re
from zhon.hanzi import punctuation
stopwords = [line.strip().replace('\n','') for line in open('../hw1/stopwords.txt','r').readlines()]
punctuations = [line.strip().replace('\n','') for line in open('../hw1/punctuation.txt','r').readlines()]
jieba.load_userdict('../hw1/userDict.txt')
punctuation = list(punctuation)

In [134]:
intro_all = []
for content in listObj:
    intro = content['intro'].replace('\n','').replace('\t','').replace('\u3000','')
    intro = re.sub('[^\u4e00-\u9fa5]','',intro)
    intro = jieba.lcut(intro)
    Intro = []
    for word in intro:
        if word not in stopwords:
            Intro.append(word)
    Intro.insert(0,content['ch_name'])
    Intro.insert(0,content['doc_id'])
    intro_all.append(Intro)

In [136]:
words = {}
for intro in intro_all:
    for word in intro:
        if word in words:
            words[word].append(intro[0])
        else:
            words[word] = [intro[0]]

In [ ]:
from collections import Counter
inverted_index = {}
for i,j in words.items():
    print(i)
    wordcount = (Counter(j))
    list(wordcount.items()).sort(key = lambda x : x[1],reverse = True)
    wordcounts = list(wordcount.items())
    inverted_index[i] = wordcounts

In [141]:
#PAGERANK
import networkx as nx
G = nx.DiGraph()
Edges = []
for word,doc_id in inverted_index.items():
    edges = []
    for ID in doc_id[2:]:
        if ID[0] != doc_id[0][0]:
            edges.append((doc_id[0][0],ID[0]))
        Edges.append(edges)

In [143]:
for edges in Edges:
    for edge in edges:
        G.add_edge(edge[0],edge[1])

In [ ]:
import networkx as nx
pagerank_list = nx.pagerank(G,alpha = 1)
print(pagerank_list)

In [ ]:
PageRankList = list(pagerank_list.items())
PageRankList.sort(key = lambda x:x[0])
print(PageRankList)

In [297]:
print(PageRankList)

[(1, 4.544930685798488e-05), (2, 4.544930685798488e-05), (3, 4.545420585719385e-05), (4, 4.544930685798488e-05), (5, 4.544930685798488e-05), (6, 4.545420585719385e-05), (7, 4.5464189899619023e-05), (8, 4.545407364496137e-05), (10, 4.545420585719385e-05), (11, 4.546401695496681e-05), (12, 4.5464189899619023e-05), (13, 4.545901337659006e-05), (14, 4.5476806337213934e-05), (15, 4.548372790478019e-05), (18, 4.547406814329251e-05), (19, 4.546374934745792e-05), (22, 4.5472866876614384e-05), (23, 4.549048992252775e-05), (24, 4.548372790478019e-05), (25, 4.548895512832042e-05), (26, 4.5519253658579766e-05), (27, 4.549766698611565e-05), (28, 4.547924331285174e-05), (29, 4.548166778527099e-05), (30, 4.5511955792749076e-05), (31, 4.555141295912916e-05), (32, 4.5504863583058874e-05), (33, 4.5518068883908946e-05), (34, 4.5526299956806095e-05), (35, 4.553928720181289e-05), (36, 4.554454274548742e-05), (37, 4.557200296118578e-05), (38, 4.554861290373865e-05), (39, 4.555365069891025e-05), (40, 4.55093

In [233]:
#CREATING DOC LINKS
DOC_LINK = {}
doc_link = []
i = 0
for data in inverted_index.values():
    id_num = intro_all[i][0] 
    doc_id = data[0][0]
    for ID in data:
        if ID[0] not in doc_link and ID[0]!= doc_id:
            doc_link.append(ID[0])
    if doc_id != id_num:
        i+=1
        doc_link.sort()
        DOC_LINK[id_num] = doc_link
        doc_link = []
    

In [298]:
#SAVING DATA into json file hw2.json
# print(dict(listObj[0]))
DATA = []
for listobj in listObj:
    try:
        d_link = DOC_LINK[dict(listobj)['doc_id']]
    except:
        d_link = "None"
    try:
        prl = dict(PageRankList)[dict(listobj)['doc_id']]
    except:
        prl = 0
    info = {
        'doc_id' : dict(listobj)['doc_id'],
        'ch_name' : dict(listobj)['ch_name'],
        'en_name' : dict(listobj)['en_name'],
        'pagerank' : prl,
        'label[class]' : dict(listobj)['label[class]'],
        'intro' : dict(listobj)['intro'],
        'release_data' : dict(listobj)['release_date'],
        'links(doc_id)' : d_link,
    }
#     print(dict(listobj)['doc_id'])
    DATA.append(info)


In [302]:
#turn data into dictionary with id as key
movie_data = {}
for movie in DATA:
    movie_data[dict(movie)['doc_id']] = movie

In [ ]:
#QUERY

In [402]:
class search_engine():
    def query(text):
        count_all = 0
        count = 0
        movie_prl = []
        movie_list = []
        #find all movie related to text
        regex = re.compile(rf'(.*){text}')
        for movie,data in movie_data.items():
            if regex.match(dict(data)['ch_name']) or regex.match(dict(data)['en_name']) or regex.match(dict(data)['intro']):
                count_all += 1
        for movie,index in inverted_index.items():
            if regex.match(dict(data)['ch_name']) or regex.match(dict(data)['en_name']) or regex.match(dict(data)['intro']):
                    count += 1
            if str(text) in str(movie):
                count +=1
                for i in index:
                    movie_list.append(i[0])
        #print all movies related with text
        movies_list=list(dict.fromkeys(movie_list))
        data = []
        for ID in movies_list:
            data.append([movie_data[ID]['doc_id'],movie_data[ID]['pagerank'],movie_data[ID]['ch_name'], movie_data[ID]['en_name'],movie_data[ID]['intro']])
        data.sort(key = lambda x:x[1],reverse = False)
        for movie in data:
            print(movie)
        print(f"Precision = "+count/count_all+100+"%")
        print(count)
        print(count_all)
        

In [403]:
search_engine.query("葉問")

[4146, 4.9803758557095485e-05, '開心魔法', 'Magic to Win', '★《葉問》導演葉偉信 魔幻傑作  新年賀歲  歡笑首選劇情大綱宇宙之大，無奇不有，有很多不可思議的事情是無法解釋。康森貴 (黃百鳴 飾)除了是一名大學教授，更是五行魔法的「水」系魔法師，卻無人知道他擁有魔法的秘密。因為一次意外，康森貴的魔法竟轉移到學生美斯 (吳千語 飾) 身上，這位平凡的少女因而捲入魔法世界的漩渦。「火」「木」「土」「金」系魔法師，齊力集合五行能量逆轉時間，制止災難發生……']
[3430, 5.044893117016784e-05, '葉問前傳', 'The Legend is Born –IP Man', '佛山童年葉問跟隨詠春宗師陳華順（洪金寶飾）門下學習詠春拳。未久，陳華順去世後，葉問即跟隨掌門師兄吳仲素（元彪飾）繼續學習詠春拳。其後葉問（杜宇航飾）選擇暫別師兄葉天賜（樊少皇飾），師妹李美慧（陳嘉桓飾）及一眾武館師兄弟，獨自到香港求學。當中葉問更被外籍人士侮辱，歧視……其後葉問學業有成，返回家鄉與佛山副市長的女兒張永成（黃奕飾）相戀而卻遇到重重障礙，那一邊廂的天賜原來一早已背負著一個重大任務。與此同時，大師兄吳仲素發現了葉問習得梁壁（葉准飾）的詠春拳而偏離陳華順所強調之正統詠春。 因門戶觀念下，兩人對是否應該堅守 “正統” 這觀唸有著重大分歧，在尊師重道及鑽研武術上是否應該另辟門徑？在這問題上，葉問在兩難局面之下如何解決 ？而天賜到最後能否完成任務？而另外葉問，如何衝破障礙與永成在一起？']
[4742, 5.104627616725772e-05, '葉問：終極一戰', 'IP MAN:FINAL FIGHT', '1949年，患有長期胃病的葉問 (黃秋生 飾) 隻身來港謀生，得一班新收徒弟照顧，在港九飯店職工總會天台開始教授詠春。其中當差的鄧聲 (陳小春 飾) 因鎮壓工人罷工，被逼與同門師弟妹梁雙 (洪天明 飾)、李瓊 (蔣璐霞 飾)、陳四妹 (鍾欣桐 飾)等為敵。期間，葉妻張永成 (袁詠儀 飾) 從佛山來港團聚，可惜香港生活太艱難，來了又還，未料夫妻從此永別。 即使生活捉襟見肘，這位一代宗師卻從未折腰，《葉問：終極一戰》還原最真實的葉問，把他在香港的中、晚年生活如實呈現。']
[2922, 5.13078360219773

TypeError: can only concatenate str (not "float") to str

In [370]:
# count = 0
# for movie,data in movie_data.items():
#     if "葉問" in str(data):
# #         movie_list = index
#         print(data)
#         count+=1
# #         print(movie_list)
# #         print(movie_data[movie_list[0][0]])
# print(count)
# for ID in movie_list:
#             movie_prl.append(movie_data[ID[0]])
# movie_prl = []
movie_list = []
# movie_sort = []
for movie,index in inverted_index.items():
            if str("葉問") in str(movie):
                for i in index:
                    a = (i[0])
                    print(a)
                    movie_list.append(a)
movies_list=list(dict.fromkeys(movie_list))
print(movie_list)
#print all movies related with text
data = []
for ID in movie_list:
    pr = dict(PageRankList)[ID]
    movie_prl.append(pr)
    data.append([movie_data[ID]['doc_id'],movie_data[ID]['pagerank'],movie_data[ID]['ch_name'], movie_data[ID]['en_name'],movie_data[ID]['intro']])
data.sort(key = lambda x:x[1],reverse = False)
for movie in data:
    print(movie)
#     list(my_dict.keys())[list(my_dict.values()).index(100)])

2922
3291
3320
3377
3430
3560
3689
4146
4382
4449
4505
4624
4742
4875
5108
5582
5770
5772
5949
6169
6290
6349
7037
7352
7700
8632
8869
9890
10675
10991
11826
14653
14821
3377
3430
4742
5949
8869
9890
10675
10991
[2922, 3291, 3320, 3377, 3430, 3560, 3689, 4146, 4382, 4449, 4505, 4624, 4742, 4875, 5108, 5582, 5770, 5772, 5949, 6169, 6290, 6349, 7037, 7352, 7700, 8632, 8869, 9890, 10675, 10991, 11826, 14653, 14821, 3377, 3430, 4742, 5949, 8869, 9890, 10675, 10991]
[4146, 4.9803758557095485e-05, '開心魔法', 'Magic to Win', '★《葉問》導演葉偉信 魔幻傑作  新年賀歲  歡笑首選劇情大綱宇宙之大，無奇不有，有很多不可思議的事情是無法解釋。康森貴 (黃百鳴 飾)除了是一名大學教授，更是五行魔法的「水」系魔法師，卻無人知道他擁有魔法的秘密。因為一次意外，康森貴的魔法竟轉移到學生美斯 (吳千語 飾) 身上，這位平凡的少女因而捲入魔法世界的漩渦。「火」「木」「土」「金」系魔法師，齊力集合五行能量逆轉時間，制止災難發生……']
[3430, 5.044893117016784e-05, '葉問前傳', 'The Legend is Born –IP Man', '佛山童年葉問跟隨詠春宗師陳華順（洪金寶飾）門下學習詠春拳。未久，陳華順去世後，葉問即跟隨掌門師兄吳仲素（元彪飾）繼續學習詠春拳。其後葉問（杜宇航飾）選擇暫別師兄葉天賜（樊少皇飾），師妹李美慧（陳嘉桓飾）及一眾武館師兄弟，獨自到香港求學。當中葉問更被外籍人士侮辱，歧視……其後葉問學業有成，返回家鄉與佛山副市長的女兒張永成（黃奕飾）相戀而卻遇到重重障礙，那一邊廂的天賜原來一早已背負著一個重大任務。與此同時，大師兄吳仲素

In [322]:
print(list(movie_prl)[0]['pagerank'])

5.130783602197735e-05
